# Building and Optimizing an Emulator

Let's build an emulator for a quantum scattering problem with an optical potential! We will choose neutron scattering on the $^{40}$ Ca nucleus as our test problem. In particular, we want to build an emulator that can predict the differential elastic scattering cross section for this reaction, at a given energy as quickly and accurately as possible. 

We have the following Hamiltonian as the radial part of the scattering equation:

\begin{equation}
    F_\alpha(\phi)=\left(-\frac{d^2}{dr^2}+\frac{\ell(\ell+1)}{r^2}+U(r,\alpha)-p^2\right)\phi(r)=0,
\end{equation}

where a system with reduced mass $\mu$ interacts through a potential $V(r,\alpha)=U(r,\alpha)/2\mu$ with parameters $\alpha$, $\ell$ is the angular momentum quantum number, and $p$ is the asymptotic linear momentum. In the code implementation we re-scale the equation to work in dimensionless units $s=rp$. Our reduced basis expansion of $n$ basis can be written as:

\begin{equation}
   \phi(r) \approx \phi_0(r) + \sum_k^n a_k\phi_k(r)
\end{equation}


The goal of this tutorial is to demonstrate how to build such an emulator with ROSE, and then choose the optimal emulator configuration to accomplish this. For a video and other notebook resources on ROSE [check this link](https://indico.cern.ch/event/1223721/contributions/5394829/). The full derivation of the involved equations, both for the high fidelity and the reduced basis counter part, will be presented in an upcoming publication and linked here.


Let's get to it then. First, we'll import all the modules we need and set up the scattering system.

In [1]:
# pip install nuclear-rose (if you don't have it already)
# pip install nuclear-rose==0.9.4a0 
# import ROSE to construct the emulator
import rose

# other nice things
import numpy as np
import time
import os
from scipy.stats import qmc

# import stuff for nice plotting
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import scipy.stats as sps
from tqdm import tqdm
import random
import datetime

plt.rcParams.update({'font.size': 12})
print(rose.__version__)

0.9.7.dev12+g8ec385d.d20230926


In [2]:
Starting_time=time.time()

## Setting up the Scattering System

We start by making some preliminary definitions for constants.

In [3]:
A = 40  # mass of the target

AMU = 931.494102  # MeV/c^2, Particle Data Group
MASS_N = 1.008665 * AMU  # MeV/c^2 PDG
MASS_P = 1.007276 * AMU  # MeV/c^2 PDG
MASS_CHARGED_PION = 139.57039  # MeV/c^2
B_40CA = 342.0522  # BMEX

MASS_40CA = 20 * MASS_P + 20 * MASS_N - B_40CA
MU = (
    MASS_40CA * MASS_N / (MASS_40CA + MASS_N)
)  # reduced mass - we will do calculations in COM frame

# asymptotic energy and wavenumber in center-of-mass (COM) frame
energy = 14.1  # MeV
k = np.sqrt(2 * MU * energy) / rose.constants.HBARC

# how many partial waves should we calculate?
l_max = 10
l_list = list(range(l_max + 1))

# domain of the differential cross section; the observable we want to emulate
angles = np.linspace(1, 179, 179)
# ROSE also has a convenient grid over radial space we can steal
rho = rose.constants.DEFAULT_RHO_MESH.copy()

## Define the Interaction: an Optical Potential

Let's define the functional forms for the interaction potentials we will use. We will use standard forms of the optical potential, with default parameters for $^{40}$Ca, as defined by [Koning and Delaroche](https://www.sciencedirect.com/science/article/pii/S0375947402013210?casa_token=qS1v6U4xDQEAAAAA:NIi9D5LpP3f05AMwRnvbQ6or8hSvXoEIgKBV56KA4l9aObCOVDAndmuCeIH77iuzoXMOOlAMyw). 

The potential we use includes the following terms, each with a set of parameters:

1. complex volume term with a Woods-Saxon shape; $V_v$, $W_v$, $R_v$, $a_v$
2. imaginary surface-peaked term with a Woods-Saxon derivative shape; $W_d$, $R_d$, $a_d$
3. complex spin-orbit (SO) coupling term with a Woods-Saxon derivative shape; $V_{so}$, $W_{so}$, $R_{so}$, $a_{so}$.


\begin{equation}
    \begin{aligned}
      &V(r;\alpha,\ell,j) =-V_v f_\text{WS}(r,R_v,a_v) - iW_v f_\text{WS}(r,R_v,a_v) \\
      &-i4a_dW_d \frac{d}{dr}f_\text{WS}(r,R_d,a_d) \\
      &+2\ell\cdot s V_{so}  \Big(\frac{\hbar}{m_\pi c}\Big)^2 \frac{1}{r} \frac{d}{dr}f_\text{WS}(R_{so},a_{so}),
    \end{aligned}
\end{equation}


The Woods-Saxon functional form (and it's derivative) is our model for the geometric shape of the effective potential the neutron feels as a function of it's distance from the $^{40}$ Ca nucleus. It looks like this:

\begin{equation}
f(r;R,a) = \frac{1}{1 - \exp{\frac{r-R}{a}}}.
\end{equation}


All depth parameters ($V_i$, $W_i$) are given in MeV, and geometric parameters $r_i$ and $a_i$ are given in fm. We define our vector of parameters as 

\begin{equation}
\boldsymbol{\alpha} = \begin{bmatrix}
V_v &  W_v & R_v & a_v & W_d & R_d & a_d & V_{so} & W_{so} & R_{so} & a_{so}
\end{bmatrix}
\end{equation}

Let's set this up:


In [4]:
# shape of interaction for volume term
def wood_saxon(r, R, a):
    return 1 / (1 + np.exp((r - R) / a))


# shape of interaction for surface-peaked and spin-orbit coupling terms
def wood_saxon_prime(r, R, a):
    return -1 / a * np.exp((r - R) / a) / (1 + np.exp((r - R) / a)) ** 2


# total potential with a real and central term (with the same geometry)
# and imaginary surface-peaked term, but no SO coupling
def optical_potential(r, theta):
    Vv, Wv, Wd, Vso, Wso, Rv, Rd, Rso, av, ad, aso = theta
    return (1j * Wv - Vv) * wood_saxon(r, Rv, av) - (4j * ad * Wd) * wood_saxon_prime(
        r, Rd, ad
    )


# spin orbit interaction constant
mso = rose.constants.HBARC / MASS_CHARGED_PION


# spin-orbit (SO) coulpling term - a function of l dot s, l being the orbital angular momentum
# and s being the spin of the neutron
def spin_orbit_potential(r, theta, ldots):
    Vv, Wv, Wd, Vso, Wso, Rv, Rd, Rso, av, ad, aso = theta
    return (Vso + 1j * Wso) * mso**2 * ldots * wood_saxon_prime(r, Rso, aso) / r


# the total number of parameters
nparams = 11

In [5]:
# the default parameters determined by Koning & Delaroche
VvKD = 46.7238
WvKD = 1.72334
WdKD = -7.2357
VsoKD = 6.1
WsoKD = -3.1

RvKD = 4.0538
RdKD = 4.4055
RsoKD = 1.01 * 40 ** (1.0 / 3.0)

avKD = 0.6718
adKD = 0.5379
asoKD = 0.60

# This is the value of the parameters coming from the Koning-Delaroche parametrization.
# Taken from https://www-nds.iaea.org/RIPL-3/
alphaCentral = np.array(
    [VvKD, WvKD, WdKD, VsoKD, WsoKD, RvKD, RdKD, RsoKD, avKD, adKD, asoKD]
)

## Set up the Training and Testing Regions for the Emulator

It is now our goal to define an emulator that, given an arbitrary point in a closed subset of this 11D space, reproduces the scattering solutions of the Schrödinger equation for the center-of-mass frame of a 14.1 MeV neutron and a $^{40}$ Ca nucleus. To do this, we will use a high-fidelity (HF) solver to construct a basis of solutions as we perturb $\boldsymbol{\alpha}$ around our region of interest, and then construct a suitable reduced basis emulator from it. 

Notice that the functional form of the Woods-Saxon (and its derivative) is not a simple linear function of the parameters $R$ and $a$. To be more precise, we can't factorize this function into the product of functions of just the parameters $R$ and $a$, and another function of just the domain of the problem $r$. A mathematician would say that this operator is not "affine" in the geometric parameters. To handle emulating the potential operator, we will need to construct an approximation that is factorizable using something called the Empirical Interplation Method (EIM). Chapters 2 and 3 of [this book](https://dr.ascsn.net/) contain a more detail view (within a nuclear physics context) of the reduced basis method and the empirical interpolation method we use in ROSE. 

In summary, the EIM consists of approximating the potential with $m$ reduced basis in the same way as we did for our solutions:

\begin{equation}
V(r; \alpha)  \approx V_\text{EIM} (r; \alpha)  = \sum_k^m \beta_k(\alpha) V_k(r).
\end{equation}

Fortunately, ROSE will handle all of these details for us, so we don't have to get our hands too dirty! Let's take a look at how to use ROSE to make ourselves a nice emulator.

We will create training boundaries 20% above and below our mean values for the parameters, and therefore define the bounds of a hyper-box in our 11D parameter space that contains this region of interst. We will train our emulator with random samples of $\alpha$ from this box, and we would like to be accurate throughout the entire box.

In [6]:
scaleTraining = 0.2

bounds = np.array(
    [
        alphaCentral - np.fabs(alphaCentral * scaleTraining),
        alphaCentral + np.fabs(alphaCentral * scaleTraining),
    ]
).T

In [7]:
def sample_points(npoints, bounds,initial_seed=None):
    sampler = qmc.LatinHypercube(d=len(bounds), seed=initial_seed)
    sample = sampler.random(npoints)
    scaled = qmc.scale(sample, bounds[:, 0], bounds[:, 1])
    return scaled


n_test = 50
n_train = 50

seed_train=142857
seed_test=142857*2

training_samples = sample_points(n_train, bounds,initial_seed=seed_train)
test_samples = sample_points(n_test, bounds,initial_seed=seed_test)

## Construct the Interaction, the High-Fidelity Solver, and the Emulator

Great, we have all the points we need to sample. Now let's define an InteractionEIMSpace in ROSE, which encodes the original and Empirically-Interpolated potential we've defined for each partial wave. Then we will use this to construct our emulators and our High-Fidelity (HF) solver, and take a look at how snapshots of the HF solutions vary over our training space.  We will use the default EIM setup. ROSE will automatically use the exact potential for the HF solver, and the EIM decomposed potential for emulation.

By the way, we like to call HF solutions of our system "snapshots". In this case, we use snapshots to refer to the test observables (differential cross sections), but we will also use it to refer to the training wavefunctions we generate with the HF solver, which ROSE uses to construct the reduced basis.

We will start by using a reduced basis of 4 elements, and the same number of EIM terms as there are parameters in $\boldsymbol{\alpha}$; in this case, 11. We will pass in the bounds of our parameter space, and the `InteractionEIMSpace` object will automatically find 1000 (by default) well-distributed points in our parameter space to train the EIM factorization. Both these numbers can be easily changed.

In [8]:
# Finally, we can invoke ROSE. Let's build an InteractionEIMSpace, running over lmax partial waves,
# with the optical potential we defined above.
interactions = rose.InteractionEIMSpace(
    optical_potential,
    nparams,
    MU,
    energy,
    l_max,
    is_complex=True,
    spin_orbit_potential=spin_orbit_potential,
    training_info=bounds,
    n_basis=11
)
# that was easy!

Now that we have our interaction, let's set up 1) a HF solver and 2) an emulator. In ROSE, these tasks are handled by the same object; `ScatteringAmplitudeEmulator`. We will set one up, and calculate snapshots of the cross sections at our test points.

In [9]:
# let's build our first reduced basis emulator
emulator = rose.ScatteringAmplitudeEmulator.from_train(
    interactions,
    training_samples,
    l_max,
    n_basis=5,
    angles=angles / 180 * np.pi,
    hf_tols=[
        10 ** (-9),
        10 ** (-9),
    ],  # these are the relative and absolute tolerances for the HF Runge-Kutta solver
)

100%|███████████████████████████████████████████| 11/11 [01:21<00:00,  7.39s/it]
/home/kyle/umich/rose/src/rose/utility.py:86: RuntimeWarning: divide by zero encountered in double_scalars
  return np.hstack([1/(2*s_c) * (3 - (s[ii]/s_c)**2), 1/s[jj]])


In [ ]:
# calculate the exact differential cross section for the 50 test parameters we sampled
test_CS = []
for params in tqdm(test_samples):
    xs = emulator.exact_xs(params)
    test_CS.append(xs.dsdo)

  8%|███▌                                        | 4/50 [00:33<06:17,  8.20s/it]

In [ ]:
# now let's plot the differential scattering cross section for each of these samples test points!
fig, ax = plt.subplots(figsize=(6, 4), dpi=300)
fig.patch.set_facecolor("white")


for i in range(n_test):
    ax.plot(angles, test_CS[i])
ax.set_yscale("log")
plt.xlim([0, 180])
plt.xlabel(r"$\theta$ [$^\circ$]")
plt.ylabel(r"$\frac{d \sigma}{d \Omega}$ [mb/sr]")
plt.show()

Great, we have our test cross section! Now let's reproduce them with our emulator.

In [ ]:
# calculate the exact differential cross section for the 50 test parameters we sampled
test_CS_emu = []
for params in tqdm(test_samples):
    xs = emulator.emulate_xs(params)
    test_CS_emu.append(xs.dsdo)

In [ ]:
fig, ax = plt.subplots(figsize=(6, 4), dpi=300)
fig.patch.set_facecolor("white")

# only plot some of them so the plot isn't too cluttered
for i in range(10):
    p = ax.plot(angles, test_CS[i])
    ax.plot(angles, test_CS_emu[i], linestyle= "dashed" ,color=p[0].get_color())

    # ax.plot(angles, test_CS[i])
    # ax.plot(angles, test_CS_emu[i], linestyle= "dashed" )

legend_styles = [
    Line2D([0], [0], color="k", linestyle="--"),
    Line2D([0], [0], color="k"),
]
ax.legend(legend_styles, ["emulated", "high-fidelity"])
ax.set_yscale("log")
plt.xlim([0, 180])
plt.xlabel(r"$\theta$ [$^\circ$]")
plt.ylabel(r"$\frac{d \sigma}{d \Omega}$ [mb/sr]")
plt.show()

This is not bad, but we can do better by optimizing the hyperparameters a bit. Many computational methods have one or more handles to trade off better accuracy at the cost of loss of speed. In the case of the reduced basis emulator we can directly change 1) the size of the reduced basis, and 2) the number of terms in the EIM expansion. Remember, we used 5 reduced basis elements, and 11 EIM terms. 

Let's write a little function that can test more configurations:

In [ ]:
def emulator_trainer(sae_config: tuple):
    r"""
    build an emulator to specification of sae_config
    Parameters:
        sae_config (size of reduced basis, number of EIM terms)
    """

    (n_basis, n_EIM) = sae_config

    interaction_space_eim = rose.InteractionEIMSpace(
        optical_potential,
        nparams,
        MU,
        energy,
        l_max,
        bounds,
        spin_orbit_potential=spin_orbit_potential,
        is_complex=True,
        n_basis=n_EIM,
    )

    emulator = rose.ScatteringAmplitudeEmulator.from_train(
        interactions,
        training_samples,
        l_max,
        n_basis=n_basis,
        angles=angles / 180 * np.pi,
        hf_tols=[
            10 ** (-9),
            10 ** (-9),
        ],
    )

    return emulator

Great, now let's choose a few configurations for the emulator and compare them. Our metrics will be:
1. time to generate solution
2. accuracy of solution (relative to the HF solver with a very small tolerance)

For each of the configs we run, we will set up the emulator, and calculate these 2 metrics. First, we will do the offline training for all the emulator configs. This is where most of the computational expense of an emulator comes in; training in the offline stage. Once each partial wave, at each paramater sample point, has it's reduced basis trained, the online stage, where we calculate observables, should be much faster.

In [ ]:
# [basis size, number of terms in EIM decomposition]
Sae_configs = [(3, 3),  (3, 5), (5,5), (7,7) ,(10, 10)]

# set up the emulators for each config
Sae_Emulators = {}

for saeconfig in Sae_configs:
    key = f"sae_{saeconfig[0]}_{saeconfig[1]}"
    Sae_Emulators[key] = emulator_trainer(saeconfig)

We would like to compare the results of our emulartors to HF solvers at a variety of tolerances, to make sure using an emulator is actually worthwhile, as opposed to just cranking down the tolerances on our HF solver. To do that, let's also set up some "dummy" `ScatteringAmplitudeEmulators`; we will just use for their HF solvers:

In [ ]:
# log of tolerance
HF_configs=[-1.5,-2,-3,-4] 

# set up the HF solvers for each config
HF_Solvers={}

for HFconfig in HF_configs:
    key = f'HF_{-HFconfig}'
    HF_Solvers[key]=rose.ScatteringAmplitudeEmulator.from_train(
        interactions,
        sample_points(2, bounds), # only 2 training points so training is quick
        l_max,
        n_basis = 2, # only 2 basis elements so trainnig is quick
        angles = angles/180*np.pi,
        hf_tols=[10**(HFconfig),10**(HFconfig)]) # we will only be using the HF solver here

We can compare a prediction comming from one of the emulators (5 basis for wave functions, 3 for EIM), one of the HF, and the original highest fidelity solver we employed as follows:

In [ ]:
emu_choice=f"sae_{Sae_configs[2][0]}_{Sae_configs[2][1]}"
HF_choice=f'HF_{-HF_configs[1]}'
print("Emulator choice: ")
print(emu_choice)
print("HF choice: ")
print(HF_choice)

In [ ]:
single_param=test_samples[0]
emulator_prediction=Sae_Emulators[emu_choice].emulate_xs(single_param).dsdo
HF_prediction=HF_Solvers[HF_choice].exact_xs(single_param).dsdo
Super_HF_prediction=test_CS[0]

In [ ]:
fig, ax = plt.subplots(figsize=(4, 3), dpi=300)
fig.patch.set_facecolor("white")

# only plot some of them so the plot isn't too cluttered

ax.plot(angles, Super_HF_prediction,label='Super HF')
ax.plot(angles, HF_prediction, label="high-fidelity "+f' ({-HF_configs[1]})',linestyle='dashed')
ax.plot(angles,emulator_prediction, label ='emulated '+ f"({Sae_configs[2][0]},{Sae_configs[2][1]})",linestyle='dashed')


ax.legend()
ax.set_yscale("log")
plt.xlim([0, 180])
plt.xlabel(r"$\theta$ [$^\circ$]")
plt.ylabel(r"$\frac{d \sigma}{d \Omega}$ [mb/sr]")
plt.rc("xtick", labelsize=10)
plt.rc("ytick", labelsize=10)
plt.show()

Great, we have trained all of our emulators and set up our HF solvers. Now let's write a function that calculates the metrics we've defined; accuracy and speed, for each sampled test parameter. We will then invoke it to get the metrics for our set of emulator configurations, and our HF solver configurations. We call this kind of analysis Computational Accuracy vs. Time (CAT) analysis, and the resulting plots, CAT plots.

In [ ]:
def CAT_Data_Maker(CS_calculator, test_parameters, test_data):
    # Both the predictions and the test_data are evaluated over all angles

    prediction_list = []
    times_list = []
    for i in range(len(test_parameters)):
        st = time.time()
        predicted = CS_calculator(test_parameters[i]).dsdo
        et = time.time()
        prediction_list.append(predicted)
        times_list.append(et - st)

    residual_list = []
    residual_list_median = []
    for i in range(len(test_parameters)):
        residual_list.append(
            np.fabs((prediction_list[i] - test_data[i])) / (test_data[i])
        )
        residual_list_median.append(np.median(residual_list[i]))

    return [times_list, residual_list_median, residual_list, prediction_list]

In [ ]:
# get times and residuals for emulation with each config
CAT_Emulator_Data = {}

for sae in tqdm(Sae_Emulators):
    CAT_Data = CAT_Data_Maker(Sae_Emulators[sae].emulate_xs, test_samples, test_CS)

    CAT_Emulator_Data[sae] = {
        "times": CAT_Data[0],
        "median_residuals": CAT_Data[1],
        "residuals_list": CAT_Data[2],
        "full_predictions": CAT_Data[3],
    }

See how quick it was to emulate the cross sections? That's what we mean by a fast online stage!

In [ ]:
# get times and residuals for the HF solver withb each config
CAT_HF_Data = {}

for HF in tqdm(HF_Solvers):
    CAT_Data = CAT_Data_Maker(HF_Solvers[HF].exact_xs, test_samples, test_CS)

    CAT_HF_Data[HF] = {
        "times": CAT_Data[0],
        "median_residuals": CAT_Data[1],
        "residuals_list": CAT_Data[2],
        "full_predictions": CAT_Data[3],
    }

Comparatively, calculating cross sections with the HF solver takes forever...

## Generate the CAT Plot

Now that we have the runtimes and residuals for each emulator and HF configuration, let's throw it all onto a nice CAT plot. A lot of the code bellow is done to create aesthetically pleasing CAT plots, you could always just plot the mean computation time vs the mean error for each emulation configuration in a simple way.

In [ ]:
#Some colors that Pablo likes:
colors = [
    "#1f77b4",
    "#ff7f0e",
    "#2ca02c",
    "#d62728",
    "#9467bd",
    "#8c564b",
    "#e377c2",
    "#7f7f7f",
    "#bcbd22",
    "#17becf",
] 

fig, ax = plt.subplots(figsize=(15, 7), dpi=400)

custom_lines = []

custom_lines.append(
    Line2D(
        [],
        [],
        color="w",
        marker="X",
        linestyle="None",
        markersize=20,
        label="[$n_\phi$, $n_U$]",
    )
)
i = 0
for sae in Sae_Emulators:
    data_sae = [
        np.array(CAT_Emulator_Data[sae]["times"]),
        np.array( [    np.median(CAT_Emulator_Data[sae]["residuals_list"][ii]) for ii in range(len(CAT_Emulator_Data[sae]["residuals_list"]))   ]  ),
    ]
    data_sae = np.array(data_sae)

  
    ax.scatter(data_sae[0], data_sae[1], s=7, color=colors[i],alpha=0.8)

    custom_lines.append(
        Line2D(
            [],
            [],
            color=colors[i],
            marker="o",
            linestyle="None",
            markersize=20,
            label=str(Sae_configs[i]),
        )
    )

    i = i + 1


i = 0
for HF in HF_Solvers:
    data_hf = [
        np.array(CAT_HF_Data[HF]["times"]),
        np.array( [    np.median(CAT_HF_Data[HF]["residuals_list"][ii]) for ii in range(len(CAT_HF_Data[HF]["residuals_list"]))   ]  ),
    ]
    data_hf = np.array(data_hf)
   
    ax.scatter(data_hf[0], data_hf[1], s=5, color="b")
    i = i + 1


custom_lines.append(
    Line2D(
        [], [], color="b", marker="o", linestyle="None", markersize=20, label="HF"
    )
)
ax.legend(handles=custom_lines, fontsize=22, frameon=True, edgecolor="black")
ax.plot([10 ** (-4), 0.036], [0.1, 0.1], color="k", linewidth=1, linestyle="dashed")
ax.plot([0.036, 0.036], [10 ** (-9), 0.1], color="k", linewidth=1, linestyle="dashed")

ax.set_xscale("log")
ax.set_yscale("log")
ax.set_xlabel("Time per sample (s)", fontsize=23)
ax.set_ylabel(
    r"Median CS Error ", fontsize=23
)
plt.rc("xtick", labelsize=23)
plt.rc("ytick", labelsize=23)

plt.xlim(5 * 10 ** (-4), 5 * 10 ** (1))
plt.ylim(10 ** (-5), 10 ** (1))

plt.show()

Great, now we can decide empirically which emulator configuration is the fastest for a given accuracy, and we can quantitatively see that the emulators are categorically superior to the HF solver when it comes to getting fast and accurate differential cross sections. The dashed black lines represent an (arbitrary) target for speed and accuracy: more than 1 million samples per hour and less than 0.1 median absolute error. Options (7,7) and (10,10) for the emulator fullfil both of these requirements, while none of the high fidelity solvers get near the target region.

Notice, increasing the number of EIM terms can increase accuracy without making too much large impact on time. We would expect diminishing returns for more EIM terms, however, and if the error is dominated by a lack of enough wave function basis (as the case of the (3,5) emulator) then increasing the EIM terms will not improve the emulator at all. For basis size, since ROSE has to actually solve a system of equations proportional to this variable, we except the time to scale roughly with the square of the basis size, so the impact on time is much larger. Since we only trained with 50 samples, in this case increasing te basis number further will saturate the median error in around 10^-4, but further improvement can be done by providing more training points for ROSE.

Let's plot a few cross sections in the test set from the (10,10) emulator and see how they look.

In [ ]:
emulator = Sae_Emulators["sae_10_10"]

fig, ax = plt.subplots(figsize=(6, 4), dpi=300)
fig.patch.set_facecolor("white")

# only plot some of them so the plot isn't too cluttered
for i in range(1):
    p = ax.plot(angles, emulator.emulate_xs(test_samples[i]).dsdo, "--", alpha=0.5)
    ax.plot(angles, test_CS[i], color=p[0].get_color(), alpha=0.5)
    ax.plot(angles, emulator.emulate_dsdo(test_samples[i]), 'k')

legend_styles = [
    Line2D([0], [0], color="k", linestyle="--", alpha=0.5),
    Line2D([0], [0], color="k", alpha=0.5),
]
ax.legend(legend_styles, ["emulated", "high-fidelity"])
ax.set_yscale("log")
plt.xlim([0, 180])
plt.xlabel(r"$\theta$ [$^\circ$]", fontsize=23)
plt.ylabel(r"$\frac{d \sigma}{d \Omega}$ [mb/sr]", fontsize=23)
plt.show()

Wow, what an improvement! And notice how fast it was. We had to make the lines see-thru here because the emulated and the HF solutions basically agree exactly, and with a speedup of almost 3 orders of magnitude on average.

In the next tutorial, we will use the 'best' emulator configuration we trained to quickly optimize the parameters in the optical model to experimental cross section data, with Bayesian uncertainty-quantification.

In [ ]:
Ending_time=time.time()

print("Total time to run the notebook (minutes):")

print(round((Ending_time-Starting_time)/60))